In [ ]:
import os

from astropy.time import Time, TimeDelta
from bokeh.models import DatetimeTickFormatter
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', logo=False)
from IPython.display import Markdown as md
import numpy as np
import pandas as pd
import yaml

from lsst_efd_client import EfdClient

%matplotlib inline

In [ ]:
client = EfdClient("ncsa_teststand_efd")

In [ ]:
start_time_str = "2020-08-27T22:00:00"
end_time_str = "2020-08-27T23:00:00"
#table_name = "lsst.sal.CCHeaderService.logevent_heartbeat"
table_name = "lsst.sal.MTM1M3.hardpointActuatorData"
#table_name = "lsst.sal.Rotator.motors"
#table_name = "lsst.sal.Hexapod.actuators"
#table_name = "lsst.sal.MTPtg.currentTargetStatus"
#table_name = "lsst.sal.MTPtg.timeAndDate"
columns = "private_seqNum"
index = None
#index = 1
start_time = Time(start_time_str, scale='tai')
end_time = Time(end_time_str, scale='tai')

In [ ]:
df = await client.select_time_series(table_name,
                                     columns,
                                     start_time,
                                     end_time,
                                     index=index)

In [ ]:
seqNums = df['private_seqNum'].values
print(seqNums.size)

In [ ]:
seq_delta = seqNums[1:] - seqNums[:-1]

In [ ]:
indexes = np.where(seq_delta > 1)
print(len(indexes[0]))

In [ ]:
for index in indexes[0]:
    print(seq_delta[index])
    print(df.index[index], df.index[index + 1])

In [ ]:
seq_delta[indexes]

In [ ]:
df1 = df.resample('600S').ohlc()

In [ ]:
#df.head()

In [ ]:
df1["Received Messages"] = df1[("private_seqNum", "close")] - df1[("private_seqNum", "open")] + 1

In [ ]:
df1.head()

In [ ]:
df1.tail()

In [ ]:
df2 = await client.select_time_series(table_name,
                                      ["private_sndStamp", "private_rcvStamp"] ,
                                      start_time,
                                      end_time,
                                      index=index)

In [ ]:
df2["clock_diff"] = df2["private_rcvStamp"] - df2["private_sndStamp"]

In [ ]:
df2["clock_diff"]

In [ ]:
dt_format = '%F %T'
formats = {'days': dt_format, 'months': dt_format, 'hours': dt_format, 'minutes': dt_format}
date_formatter = DatetimeTickFormatter(**formats)
tick_rotation = 75

table = hv.Table(df2.reset_index())
x_tuple = ('index', 'clock_diff')
clock_diff = hv.Curve(table, x_tuple, ("clock_diff")).opts(xlabel="Time", ylabel="Rcv - Snd (s)")
clock_diff.opts(opts.Curve(height=400, width=800, xformatter=date_formatter, xrotation=tick_rotation))

In [ ]:
max_t = np.round(np.max(df2["clock_diff"].values), decimals=3)
nbins = int(max_t / 0.0005)
hist = np.histogram(df2["clock_diff"].values, bins=np.linspace(0, max_t, nbins))
min_y = np.min(hist[0])
max_y = np.max(hist[0])
delta_t = hv.Histogram(hist, kdims=hv.Dimension('time differences', 
                                                label='Rcv - Snd', unit='s'),
                       label='Distribution of 𝚫T')
delta_t.opts(logy=True, ylim=(min_y, max_y))

In [ ]:
hist[0]